In [ ]:
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from dotenv import load_dotenv
import os
load_dotenv()
groq_Api_Key = os.environ.get("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq

# Debugging: Check if the API key is loaded correctly
if not groq_Api_Key:
    raise ValueError("GROQ_API_KEY is not set. Please check your .env file.")

llm = ChatGroq(
    groq_api_key=groq_Api_Key,
    model="openai/gpt-oss-20b"
)

prompt = "What is your name?"

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(State: State) -> State:
    print("Input Prompt (State):", State)
    response = llm.invoke(State["messages"])
    # print("LLM Response:", response)
    return {"messages": response}

builder = StateGraph(State)

builder.add_node("chatBotNode", chatbot)

builder.add_edge(START, "chatBotNode")
builder.add_edge("chatBotNode", END)

# Debugging: Compile the graph and log any issues
try:
    graph = builder.compile()
except Exception as e:
    raise RuntimeError("Error compiling the graph:", e)

# Debugging: Invoke the graph and log the output
try:
    output = graph.invoke({"messages": [prompt]})
    print("Graph Output:", output["messages"])
except Exception as e:
    raise RuntimeError("Error invoking the graph:", e)

Input State: {'messages': ['What is your name?']}
Graph Output: content='I’m ChatGPT, your AI assistant. How can I help you today?' additional_kwargs={'reasoning_content': 'The user asks "What is your name?" The assistant should respond politely, maybe say "I\'m ChatGPT, your AI assistant." Also mention name is ChatGPT. Let\'s comply.'} response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 76, 'total_tokens': 138, 'completion_time': 0.055341064, 'prompt_time': 0.004972552, 'queue_time': 0.045377628, 'total_time': 0.060313616}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_af4b447bfc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--edebff93-9937-48d7-9656-24ab3f937b14-0' usage_metadata={'input_tokens': 76, 'output_tokens': 62, 'total_tokens': 138}
Graph Output: content='I’m ChatGPT, your AI assistant. How can I help you today?' additional_kwargs={'reasoning_content': 'The user asks "What is your name?" The assistan

In [ ]:
state = None
while(True):
    try:
        in_message = input("User : ")
        if in_message.lower() in {"quit","exit"}:
            break
        if state is None:
            state = {"messages" : in_message}
        else:
            state["messages"].append(in_message)
    
        graph.invoke(state)
        print("bot : " + state["messages"][-1].content)
    except Exception as e:
        print(f" an error occured : {e}")
        break
    